In [1]:
import networkx as nx
import pickle
from torch.nn.utils.rnn import pad_sequence,pad_packed_sequence
import torch 
import sys
import numpy as np
from utils import load_file
import os
from itertools import combinations
sys.path.append('../data/pengpai/')
from address import address
HEAD = 0
TAIL = 1

In [2]:
def load_data():

    with open('old2new_case_id.pickle','rb') as file:
        old2new_case_id = pickle.load(file)
        file.close()
    with open('spread_net.pickle','rb') as file:
        spread_net = pickle.load(file)
        file.close()

    reindexed_sp_net = nx.Graph()
    for edge in spread_net.edges():
        reindexed_sp_net.add_edge(old2new_case_id[edge[HEAD]],old2new_case_id[edge[TAIL]])
    with open('../data/pengpai/labeled_data/case_path.pickle','rb') as file:
        case_path = pickle.load(file)
        file.close()
    return old2new_case_id,spread_net,old2new_case_id,reindexed_sp_net,case_path

In [3]:
def inter(list1,list2):
    return list(set(list1)&set(list2))

In [4]:
def association_analysis_full(caseid_1,caseid_2):
    try:
        path1 = [addr.get_addr('full') for addr in case_path[old2new_case_id.inv[caseid_1]]]
        path2 = [addr.get_addr('full') for addr in case_path[old2new_case_id.inv[caseid_2]]]
        inter_path = inter(path1,path2)
    except KeyError:
        return False
    if inter_path:
        return True
    else:
        return False

In [13]:
def get_path(caseid,case_path,old2new_case_id):
    path = []
    for addr in case_path[old2new_case_id.inv[caseid]]:
        path.append(addr.get_addr('full'))
    return path

In [6]:
def association_analysis_no_province(caseid_1,caseid_2):
    path1,path2 = [],[]
    try:
        path1 = get_path(caseid_1)
        path2 = get_path(caseid_2)
        inter_path = inter(path1,path2)
    except KeyError:
        return False
    if inter_path:
        return True
    else:
        return False

In [10]:
def statistics_analysis(reindexed_sp_net):

    accurancy,precision,recall = {},{},{}
    # for i in range(0,20):
    #     ratio = i/10.0
    all_pair = load_file(f'dataset/case_idx_pair/test_dataloader_4_random.pickle').dataset

    correct,wrong = 0,0
    TP,TN,FP,FN = 0.0,0.0,0.0,0.0
    for pair in all_pair:
        # if type=='full':
        association_analysis = association_analysis_full(pair[HEAD],pair[TAIL])
        # if type=='no province':
        #     association_analysis = association_analysis_no_province(pair[HEAD],pair[TAIL])
        if association_analysis==True:
            if reindexed_sp_net.has_edge(pair[HEAD],pair[TAIL]) == True:
                TP+=1
            if reindexed_sp_net.has_edge(pair[HEAD],pair[TAIL]) ==False:
                FP+=1
        else:
            if reindexed_sp_net.has_edge(pair[HEAD],pair[TAIL]) ==True:
                FN+=1
            if reindexed_sp_net.has_edge(pair[HEAD],pair[TAIL]) ==False:
                TN+=1

    accurancy = (TN+TP)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    print('TP:%d\tFP:%d\tFN:%d\tTN:%d\taccurancy:%.4f\tprecision:%.4f\trecall:%.4f\tf1:%.4f  ' % (TP,FP,FN,TN,accurancy,precision,recall,f1))
    return accurancy,precision,recall,f1

In [11]:
old2new_case_id,spread_net,old2new_case_id,reindexed_sp_net,case_path = load_data()
statistics_analysis(reindexed_sp_net)

ModuleNotFoundError: No module named 'dataloader.dataloader'; 'dataloader' is not a package